In [2]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [10]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [11]:
!pwd

/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset


In [12]:
!cat /mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_XMLs_100Snippet/V_DRONE_0013_001.xml

<annotation>
	<folder>FINAL_DATABASE_V/data_V/</folder>
	<filename>V_DRONE_0013_001.png</filename>
	<path>FINAL_DATABASE_V/data_V/V_DRONE_0013_001.png</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>640</width>
		<height>512</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>DRONE</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>124</xmin>
			<ymin>22</ymin>
			<xmax>200</xmax>
			<ymax>62</ymax>
		</bndbox>
	</object>
</annotation>


In [13]:
print(extract_info_from_xml('/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_XMLs_100Snippet/V_DRONE_0013_001.xml'))

{'bboxes': [{'class': 'DRONE', 'xmin': 124, 'ymin': 22, 'xmax': 200, 'ymax': 62}], 'filename': 'V_DRONE_0013_001.png', 'image_size': (640, 512, 3)}


In [14]:
class_name_to_id_mapping = {"DRONE":"DRONE"}
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_TXTs_100Snippet", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))


In [16]:
# Get the annotations
annotations = [os.path.join('/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_XMLs_100Snippet', x) for x in os.listdir('/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_XMLs_100Snippet') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_TXTs_100Snippet', x) for x in os.listdir('/mnt/DATA/Abhay/Downloads/Gireesh/drone/DroneTrainDataset/Drone_TXTs_100Snippet') if x[-3:] == "txt"]

100%|██████████| 17252/17252 [00:01<00:00, 12177.86it/s]
